In [409]:
import pandas as pd
citibike = pd.read_csv('data/citibike-tripdata.csv', sep=',')

# Задание 6.1 

Сколько пропусков в столбце start station id?

In [410]:
citibike['start station id'].isna().sum()

169

# Задание 6.2

In [411]:
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

# Задание 6.3 

Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.

In [412]:
citibike['start station id'].value_counts().index[0]

281.0

# Задание 6.4 

Велосипед с каким идентификатором является самым популярным?

In [413]:
citibike['bikeid'].value_counts().index[0]

33887

# Задание 6.5 

Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.

In [414]:
round(citibike['usertype'].value_counts(normalize=True).iloc[0], 2)

0.77

# Задание 6.6 

Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.

In [415]:
citibike['gender'].value_counts().iloc[0]

183582

# Задание 6.7

Выберите утверждения,которые соответствуют нашим данным:

A Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу

B В рассматриваемые дни минимальный возраст клиента составлял 10 лет

C Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave"

D Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail" 

In [416]:
len(citibike['start station id'].unique()) != len(citibike['end station id'].unique())

True

In [417]:
citibike['starttime'] = pd.to_datetime(citibike['starttime'])
citibike['stoptime'] = pd.to_datetime(citibike['stoptime'])
citibike['stoptime'].dt.year.max() - citibike['birth year'].max() == 10

False

In [418]:
citibike['start station name'].value_counts().index[-1] == 'Eastern Pkwy & Washington Ave'

True

In [419]:
citibike['end station name'].value_counts().index[1] == 'Liberty Light Rail'

False

# Задание 6.8 

В первую очередь удалим лишнюю информацию из данных.

В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки. Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?

In [420]:
citibike = citibike.drop(['start station id', 'end station id'], axis=1)
citibike.shape[1]

12

# Задание 6.9 

Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?

In [421]:
citibike['age'] = 2018 - citibike['birth year']
citibike = citibike.drop(['birth year'], axis=1)
citibike[citibike['age'] > 60].shape[0]

11837

# Задание 6.10 

Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки (stoptime) и её началом (starttime). Сколько целых минут длилась поездка под индексом 3 в таблице?

In [422]:
citibike['trip duration'] = (citibike['stoptime'] - citibike['starttime']).dt.total_seconds()/60
int(citibike['trip duration'][3])

7

# Задание 6.11 

Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.

In [423]:
citibike['weekend'] = citibike['starttime'].dt.weekday.apply(lambda x: 1 if x in [5,6] else 0)
citibike[citibike['weekend'] == 1].shape[0]

115135

# Задание 6.12 

Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:

 - поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
 - поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
  - поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
 - поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).

Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.

In [424]:
def get_tod(hour):
    if 0 <= hour <= 6:
        return 'night'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'day'
    elif 18 <= hour <= 23:
        return 'evening'

In [425]:
citibike['time_of_day'] = citibike['starttime'].dt.hour.apply(get_tod)
round(citibike[citibike['time_of_day'] == 'day'].shape[0] / citibike[citibike['time_of_day'] == 'night'].shape[0])

9